In [14]:
import sqlite3
from pathlib import Path
from typing import List, Tuple, Optional
from contextlib import contextmanager


class DBManager:
    def __init__(self, db_path: str):
        self.db_path = db_path

    @contextmanager
    def get_conn(self):
        conn = None
        try:
            conn = sqlite3.connect(self.db_path)
            yield conn.cursor()
        finally:
            if conn:
                conn.close()

    def init_db(self):
        with self.get_conn() as cur:
            cur.executescript("""
                DROP TABLE IF EXISTS rib;
                DROP TABLE IF EXISTS name;
                DROP TABLE IF EXISTS question;
                DROP TABLE IF EXISTS actual_state;

                CREATE TABLE IF NOT EXISTS rib (
                    start INTEGER,
                    end INTEGER,
                    answer TEXT,
                    PRIMARY KEY (start, end)
                );

                CREATE TABLE IF NOT EXISTS name (
                    state INTEGER PRIMARY KEY,
                    name TEXT
                );

                CREATE TABLE IF NOT EXISTS question (
                    state INTEGER PRIMARY KEY,
                    question TEXT
                );

                CREATE TABLE IF NOT EXISTS actual_state (
                    state INTEGER PRIMARY KEY
                );
            """)
            cur.connection.commit()

    def fill_data(self):
        rib_data = [
            (0, 1, "Стебель - зеленый"),
            (0, 2, "Стебель - древесный"),
            (2, 3, "Положение - стелящееся"),
            (2, 4, "Положение - прямое"),
            (4, 5, "Один ствол - да"),
            (4, 6, "Один ствол - нет"),
            (5, 7, "Форма листа широкая и плоская - да"),
            (5, 8, "Форма листа широкая и плоская - нет"),
            (8, 9, "Форма листа - чешуеобразная"),
            (8, 10, "Форма листа - иглоподобная"),
            (10, 11, "Конфигурация - хаотическая"),
            (10, 12, "Конфигурация - 2 ряда"),
            (12, 11, "Серебряная полоса - да"),
            (12, 13, "Серебряная полоса - нет"),
        ]

        name_data = [
            (1, "Тип - травянистые"),
            (3, "Тип - лианы"),
            (5, "Тип - деревья"),
            (6, "Тип - кустарниковые"),
            (7, "Класс - покрытосеменные"),
            (8, "Класс - голосеменные"),
            (9, "Семейство - кипарисовые"),
            (11, "Семейство - сосновые"),
            (13, "Семейство - болотный кипарис"),
        ]

        question_data = [
            (0, "Какой стебель у растения - древесный или зелёный?"),
            (1, "Ответ: тип – травянистые. Вопросов больше нет."),
            (2, "Какое положение стебля - стелящееся или прямостоящее?"),
            (3, "Ответ: тип – лианы. Вопросов больше нет."),
            (4, "Имеет ли растение один основной ствол?"),
            (5, "Имеют ли растения широкую и плоскую форму листа?"),
            (6, "Ответ: тип – кустарниковые. Вопросов больше нет."),
            (7, "Ответ: класс – покрытосеменные. Вопросов больше нет."),
            (8, "Какова форма листа: чешуеобразная или иглоподобная?"),
            (9, "Ответ: семейство – кипарисовые. Вопросов больше нет."),
            (10, "Какова конфигурация расположения игл: хаотическая или в 2 ряда?"),
            (11, "Ответ: семейство – сосновые. Вопросов больше нет."),
            (12, "Имеется ли серебристая полоса снизу иглы?"),
            (13, "Ответ: семейство – болотный кипарис. Вопросов больше нет."),
        ]

        with self.get_conn() as cur:
            cur.executemany("INSERT INTO rib VALUES (?, ?, ?)", rib_data)
            cur.executemany("INSERT INTO name VALUES (?, ?)", name_data)
            cur.executemany("INSERT INTO question VALUES (?, ?)", question_data)
            cur.execute("INSERT INTO actual_state VALUES (0)")
            cur.connection.commit()

    def execute_query(self, query: str, params: tuple = ()) -> List:
        with self.get_conn() as cur:
            return cur.execute(query, params).fetchall()

    def execute_update(self, query: str, params: tuple = ()):
        with self.get_conn() as cur:
            cur.execute(query, params)
            cur.connection.commit()


class ExpertSystem:
    def __init__(self, db: DBManager):
        self.db = db

    def get_question(self, state: int) -> Optional[str]:
        result = self.db.execute_query("SELECT question FROM question WHERE state = ?", (state,))
        return result[0][0] if result else None

    def get_options(self, state: int) -> List[Tuple[int, str]]:
        return self.db.execute_query("SELECT end, answer FROM rib WHERE start = ?", (state,))

    def get_state(self) -> int:
        return self.db.execute_query("SELECT state FROM actual_state")[0][0]

    def set_state(self, state: int):
        self.db.execute_update("UPDATE actual_state SET state = ?", (state,))

    def reset(self):
        self.set_state(0)

    def run(self):
        print("Начало программы\n")

        while True:
            state = self.get_state()
            question = self.get_question(state)

            if not question:
                print("Вопрос не найден")
                break

            print(question)

            options = self.get_options(state)
            if not options:
                print("\nКонец программы")
                break

            for i, (_, text) in enumerate(options, 1):
                print(f"{i}. {text}")

            choice = input("\nВыберите номер ответа (или 'q' для выхода): ").strip()

            if choice.lower() == 'q':
                print("\nКонец программы")
                break

            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(options):
                    next_state = options[choice_idx][0]
                    self.set_state(next_state)
                    print("\n" + "-" * 60 + "\n")
                else:
                    print("Некорректный номер\n")
            except ValueError:
                print("Некорректный номер\n")


def main():
    db_path = "/content/test.db"

    db = DBManager(db_path)
    db.init_db()
    db.fill_data()
    print("INFO: DB is init\n")

    system = ExpertSystem(db)

    while True:
        system.run()

        restart = input("\nНачать заново? (да/нет): ").strip().lower()
        if restart in ['да', 'д', 'yes', 'y']:
            system.reset()
            print("\n" + "=" * 60 + "\n")
        else:
            break


if __name__ == "__main__":
    main()

INFO: DB is init

Начало программы

Какой стебель у растения - древесный или зелёный?
1. Стебель - зеленый
2. Стебель - древесный

Выберите номер ответа (или 'q' для выхода): 2

------------------------------------------------------------

Какое положение стебля - стелящееся или прямостоящее?
1. Положение - стелящееся
2. Положение - прямое

Выберите номер ответа (или 'q' для выхода): 2

------------------------------------------------------------

Имеет ли растение один основной ствол?
1. Один ствол - да
2. Один ствол - нет

Выберите номер ответа (или 'q' для выхода): 1

------------------------------------------------------------

Имеют ли растения широкую и плоскую форму листа?
1. Форма листа широкая и плоская - да
2. Форма листа широкая и плоская - нет

Выберите номер ответа (или 'q' для выхода): 2

------------------------------------------------------------

Какова форма листа: чешуеобразная или иглоподобная?
1. Форма листа - чешуеобразная
2. Форма листа - иглоподобная

Выберите н